# Spectrum Analysis

#### This piece of code does the following so far:

1. Reads spectrum data in ASCII format.

2. Finds all "peaks" in the spectrum, consistent with a given flux threshold and line width. The desired prominence of the peaks can also be adjusted, if necessary. 

3. Identifies N most significant peaks

4. Reads CASSIS line catalog.txt file for one or multiple molecule(s).

5. Compares detected peaks in the spectrum with CASSIS data to identify the lines.

In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.signal as sig

%matplotlib qt5
plt.style.use('default')

### Obtain telescope spectrum data

In [112]:
# LOAD ASCII FILE
data = np.loadtxt("181_WSW_FTS200_3mm_average_data_Tmb.dat")
freqs = data[:,0][data[:,1]!=-1000] # FREQUENCY ARRAY 
temps = data[:,1][data[:,1]!=-1000] # TEMPERATURE ARRAY 


# UNITS
freq_units = "MHz"
flux_units = "K"

threshold = 5*5e-3 

# FIND DATA ABOVE THRESHOLD
passfreqs = freqs[temps>threshold]
passtemps = temps[temps>threshold]

# LINE WIDTH
width = 0.5 #(MHz)

### Find peaks

In [4]:
peaks = sig.find_peaks(temps, height=threshold, distance=2*width)[0]
peaktemps = temps[peaks]
peakfreqs = freqs[peaks]

# Save N strongest lines separately
N = 100
maxtemps = sorted(peaktemps, reverse=True)[0:N]
maxfreqs = [peakfreqs[list(peaktemps).index(i)] for i in maxtemps]

### Visualize Probable Emission Lines

In [154]:
plt.scatter(passfreqs, passtemps, s=5, c="grey")
plt.plot(freqs[temps>0], temps[temps>0], c="black", linewidth=0.5)
plt.vlines(peakfreqs, ymin=-0.2, ymax=0, colors="lime" , linewidth=0.5, label="Probable Emission Lines")
plt.vlines(maxfreqs, ymin=-0.2, ymax=0, colors="red" , linewidth=1, label="Strongest {} Emission Lines".format(N))
plt.scatter(peakfreqs, peaktemps, s=50, c="green", marker="*", label="Peaks")

plt.xlabel("Frequency ({})".format(freq_units), fontsize=15)
plt.ylabel("Temperature ({})".format(flux_units), fontsize=15)

plt.ylim([-0.2,2])
plt.legend()

plt.show()

## Hunt for Molecules

In [53]:
def separation(f1, f2):
    return abs(f1-f2)

def hunt(molecule, molecule_file):

    # LOAD CASSIS CATALOGUE
    molecule_freqs = pd.read_csv(molecule_file, sep="\t", usecols=[2])
    molecule_freqs = molecule_freqs["Frequency(Ghz)"]
    
    molecule_names = molecule_freqs.keys()
    molecule_names = [m.split()[0] for m in molecule_names]

    detections = []
    
    # COMPARE PEAKS WITH CASSIS CATALOGUE
    for i,f1 in enumerate(molecule_freqs):
        for j,f2 in enumerate(peakfreqs):
            if separation(f1, f2) < 0.5*width:
                detections.append([molecule_names[i], f1])
                
    
                
    return detections

In [172]:
molecule = "Organics"
molecule_file = "COMsCatalog.txt"

detections = hunt(molecule, molecule_file)
            
print("Found {} probable emission lines corresponding to {}".format(len(detections), molecule))

Found 233 probable emission lines corresponding to Organics


In [179]:
fig, ax = plt.subplots(constrained_layout=True)

ax.step(freqs[temps>0], temps[temps>0], c="black", linewidth=0.5)

ax.set_ylabel("Temperature ({})".format(flux_units), fontsize=15)

ax.set_ylim([-0.2,2])

ax.vlines([d[1] for d in detections], ymin=-0.2, ymax=0, colors="blue" , linewidth=1, label="Probable Detections")



secax = ax.secondary_xaxis('top')
secax.set_xlabel("Frequency ({})".format(freq_units), fontsize=15)

ax.set_xticks([d[1] for d in detections])
ax.set_xticklabels([d[0] for d in detections], rotation=75, fontsize=10)
#plt.xticks([d[1] for d in detections], labels=[d[0] for d in detections], rotation=75, fontsize=5)
#plt.yticks([])
plt.show()

In [177]:
[d[0] for d in detections]

['DCO+',
 'HC7N',
 'C2H3CN',
 'CH3COOH,',
 'C3H7CN',
 'CH3COOH,',
 'CH3COOH,',
 'CH3COOH,',
 'CH3COOH,',
 'CCS',
 'CH3COCH3',
 'CH3COCH3',
 'CH3COCH3',
 'CH3COOH,',
 'HC3N,',
 'OCS',
 'C2H5OH,v=0',
 'C3H7CN',
 'CH3CN',
 'CH3COOH,',
 'CH3CN',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'HC5N,',
 'CCCS',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'C4H',
 'C4H',
 'C4H',
 'C4H',
 'CH3CHO',
 'HC5N,',
 'CCS',
 'CH3CHO',
 'CH3OH,',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'C2H3CN',
 'OCS',
 'HCS+',
 'C4H',
 'C4H',
 'C4H',
 'C4H',
 'SO',
 'CCS',
 'CCCS',
 'CH3C3N',
 'HNCO',
 'HCN',
 'HCN',
 'HCN',
 'CH3COOH,',
 'HCO+',
 'HNC',
 'CCS',
 'HC3N,',
 'CH3CN',
 'CH3CN',
 'CCCS',
 'CH3NH2',
 'CH3COCH3',
 'CCS',
 'C4H',
 'C4H',
 'C4H',
 'C4H',
 'CH3CHO',
 'CH3CHO',
 'CH3OH,',
 'CH3OH,',
 'CH3OH,',
 'CS',
 'CCCS',
 'SO',
 'HC3N,',
 'CH2CO',
 'CH2CO',
 'H2CS',
 'CH2CO',
 'l-C3H2',
 'H2CS',
 'CCS',
 'CCCS',
 'C2H5CN',
 'H2CS',
 'C4H',
 'C4H',
 'C4H',
 'C4H',
 'l-C3H2',
 'CCS',
 'CH3OH,',
 'HC3N,',
 'SO',
 'O